In [1]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")

from langfuse.callback import CallbackHandler

langfuse_handler = CallbackHandler(
    public_key=os.environ["LANGFUSE_PUBLIC_KEY"],
    secret_key=os.environ["LANGFUSE_SECRET_KEY"],
    host=os.environ["LANGFUSE_HOST"],
)

### Initializing the LLM Model

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [4]:
out = llm.invoke("What is the capital of France?")
out.content

'The capital of France is Paris.'

### Using Chains

In [5]:
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=30)
prompt = PromptTemplate.from_template(
    "What is a good name for a company that makes {product}?"
)

chain = prompt | llm
out = chain.invoke(input={"product": "software"})

In [6]:
out.content

'Choosing a name for a software company can depend on various factors, including your target audience, the type of software you create, and your brand identity.'

### Adding Conversation Memory

#### No Memory

In [7]:
from langchain_core.messages import HumanMessage

llm.invoke(
    input=[HumanMessage(content="Hi! I'm Bob")],
    config={"callbacks": [langfuse_handler]},
)

AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-8a696b43-994c-45d2-8ecf-180b7fb74ee0-0', usage_metadata={'input_tokens': 11, 'output_tokens': 11, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Unexpected error occurred. Please check your request and contact support: https://langfuse.com/support.


In [8]:
llm.invoke(
    input=[HumanMessage(content="What is my name?")],
    config={"callbacks": [langfuse_handler]},
)

AIMessage(content="I'm sorry, but I don't have access to personal information about you unless you share it with me. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 12, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-e2255f19-4535-478d-9b01-12901bd5fb89-0', usage_metadata={'input_tokens': 12, 'output_tokens': 28, 'total_tokens': 40, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Unexpected error occurred. Please check your request and contact support: https://langfuse.com/support.
Unexpected error occurred. Please check your request and contact support: https://langfuse.com/support.


#### Using Langgraph's persistence

(https://langchain-ai.github.io/langgraph/concepts/persistence/)

In [3]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}


## Example Input to call model:
# state = {
#     "messages": [
#         {"type": "human", "content": "Hi there!"},
#         {"type": "ai", "content": "Hello! How can I help?"},
#         {"type": "human", "content": "Tell me about LangChain."}
#     ]
# }

## The response would contain the previous messages as well as the newly generated message
# response = [
#     {"type": "human", "content": "Hi there!"},
#     {"type": "ai", "content": "Hello! How can I help?"},
#     {"type": "human", "content": "Tell me about LangChain."},
#     {"type": "ai", "content": "LangChain is a framework..."}  # New message
# ]


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [5]:
# Thread ID is needed to track the conversations

config = {"configurable": {"thread_id": "test_v0"}}

In [8]:
from langchain_core.messages import HumanMessage

human_query = "Good morning, my name is Vijay!"

input_messages = [HumanMessage(content=human_query)]

out = app.invoke(input={"messages": input_messages}, config=config)

In [10]:
human_query_2 = "What is my name?"

out_2 = app.invoke(
    input={"messages": [HumanMessage(content=human_query_2)]}, config=config
)

In [16]:
out_2["messages"]

[HumanMessage(content='Good morning, my name is Vijay!', additional_kwargs={}, response_metadata={}, id='5e5ce275-132a-4893-976f-33020178d13f'),
 AIMessage(content='Good morning, Vijay! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 15, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-043f24be-d446-475b-8465-407555936626-0', usage_metadata={'input_tokens': 15, 'output_tokens': 13, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata